In [13]:
import os
import requests
import shutil
import random
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image, ImageEnhance
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# ==============================================================================
# ⚙️ PARTE 1 - PREPARACIÓN DE DATOS (Simulación de Fotogramas de Vídeo)
# ==============================================================================

# URLs públicas de ejemplo (actualizadas para mayor fiabilidad)
image_urls_base = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Dog_standing_in_snow.jpg/1280px-Dog_standing_in_snow.jpg", # Perro
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_March_2010-1.jpg/1280px-Cat_March_2010-1.jpg", # Gato
    "https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Retriever_in_water.jpg/1280px-Retriever_in_water.jpg", # Perro en agua
    "https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Seaside_Railway_Station%2C_Ryde_Pier_Head_%28107774%29.jpg/1280px-Seaside_Railway_Station%2C_Ryde_Pier_Head_%28107774%29.jpg", # Tren
    "https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Car_with_a_blue_sky.jpg/1280px-Car_with_a_blue_sky.jpg", # Coche
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/2019_Toyota_Corolla_Icon_Tech_VVT-i_1.8_Front.jpg/1280px-2019_Toyota_Corolla_Icon_Tech_VVT-i_1.8_Front.jpg", # Coche (frontal)
    "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/Vegan_food_at_restaurant_%28Pexels%29.jpg/1280px-Vegan_food_at_restaurant_%28Pexels%29.jpg", # Comida (plato)
    "https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Pizza_with_peperoni.jpg/1280px-Pizza_with_peperoni.jpg" # Pizza
]

# Directorio base para guardar los fotogramas simulados
base_frames_dir = "simulated_video_frames"
os.makedirs(base_frames_dir, exist_ok=True)

def download_and_augment_frames(urls, video_name, num_variations=2):
    """
    Descarga imágenes y crea variaciones para simular fotogramas de un vídeo.
    Guarda los fotogramas en una subcarpeta bajo base_frames_dir.
    """
    video_frames_dir = os.path.join(base_frames_dir, video_name)
    os.makedirs(video_frames_dir, exist_ok=True)

    # Añadir un User-Agent a los headers de la solicitud para evitar errores 403 Forbidden
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    print(f"\n📥 Descargando y preparando fotogramas para {video_name}...")
    frame_counter = 1
    for url in urls:
        filename = f"frame_{frame_counter:04d}.jpg"
        filepath = os.path.join(video_frames_dir, filename)
        try:
            # Pasar los headers a la solicitud get
            response = requests.get(url, stream=True, headers=headers, timeout=10) # Añadir timeout
            if response.status_code == 200:
                with open(filepath, 'wb') as out_file:
                    shutil.copyfileobj(response.raw, out_file)

                # Crear variaciones (aumento de datos)
                image = Image.open(filepath).convert("RGB")
                for v in range(num_variations):
                    # Asegurar que las variaciones tienen un nombre de archivo único
                    new_filename = f"frame_{frame_counter:04d}_var{v+1}.jpg"
                    variation = augment_image(image)
                    variation.save(os.path.join(video_frames_dir, new_filename))
                frame_counter += 1
            else:
                print(f"  ❌ Error descargando {url}: {response.status_code} {response.reason}")
        except requests.exceptions.RequestException as e:
            print(f"  ❌ Error de solicitud para {url}: {e}")
        except Exception as e:
            print(f"  ❌ Error general al procesar {url}: {e}")
    print(f"✅ Preparación de fotogramas para {video_name} completada.")

def augment_image(image):
    """Aplica transformaciones simples a una imagen para aumento de datos."""
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.8, 1.2)) # Rango más conservador
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(random.uniform(0.8, 1.2))
    return image.transpose(Image.FLIP_LEFT_RIGHT)

# ==============================================================================
# 🤖 PARTE 2 - CLASIFICADOR DE OBJETOS (Pre-entrenado en ImageNet)
# ==============================================================================

# Transformaciones necesarias para las imágenes antes de pasarlas al modelo
transform = transforms.Compose([
    transforms.Resize((224, 224)), # ResNet50 espera imágenes de 224x224
    transforms.ToTensor(),         # Convierte la imagen a un tensor de PyTorch
    transforms.Normalize(          # Normaliza con los valores estándar de ImageNet
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Cargar el modelo ResNet50 pre-entrenado en ImageNet
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.eval() # Poner el modelo en modo de evaluación (desactiva dropout, etc., para inferencia)

# Descargar la lista de las 1000 clases de ImageNet si no existe
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
labels_path = "imagenet_classes.txt"
if not os.path.exists(labels_path):
    print("⬇️ Descargando etiquetas de ImageNet...")
    r = requests.get(LABELS_URL)
    with open(labels_path, 'w') as f:
        f.write(r.text)
    print("✅ Etiquetas descargadas.")

with open(labels_path) as f:
    all_imagenet_classes = [line.strip() for line in f.readlines()]
print(f"Total de clases de ImageNet cargadas: {len(all_imagenet_classes)}")

# ==============================================================================
# 🧠 PARTE 3 - INFERENCIA DE OBJETOS EN FOTOGRAMAS
# ==============================================================================

def classify_frames_in_folder(folder_path, classes_list):
    """
    Clasifica los objetos en cada fotograma dentro de una carpeta dada.

    Args:
        folder_path (str): Ruta al directorio que contiene los fotogramas (imágenes JPG).
        classes_list (list): Lista de todas las clases posibles del modelo (ej. ImageNet).

    Returns:
        pd.DataFrame: Un DataFrame con 'fotograma', 'clase_detectada' y 'confianza'.
                      Retorna un DataFrame vacío si la carpeta no contiene imágenes.
    """
    results = []
    # Obtener y ordenar los archivos para mantener el orden de los fotogramas
    files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

    if not files:
        print(f"⚠️ No se encontraron imágenes en {folder_path}. Saltando clasificación.")
        return pd.DataFrame(columns=["fotograma", "clase_detectada", "confianza"])

    print(f"  🚀 Iniciando inferencia en {len(files)} fotogramas en {folder_path}...")
    for i, filename in enumerate(files):
        try:
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('RGB')
            input_tensor = transform(image).unsqueeze(0) # Añadir dimensión de batch

            with torch.no_grad(): # Desactivar cálculo de gradientes
                output = model(input_tensor)
                probs = torch.nn.functional.softmax(output[0], dim=0)
                confidence, pred_class_idx = torch.max(probs, 0)

            # Usamos el índice i+1 como número de fotograma para esta secuencia específica
            frame_number = i + 1
            class_name = classes_list[pred_class_idx]

            results.append({
                "fotograma": frame_number,
                "clase_detectada": class_name,
                "confianza": float(confidence)
            })
        except Exception as e:
            print(f"  ⚠️ Error procesando {filename} en {folder_path}: {e}")

    df = pd.DataFrame(results).sort_values(by="fotograma").reset_index(drop=True)
    return df

# ==============================================================================
# 📊 PARTE 4 - SIMULACIÓN DE MÚLTIPLES VÍDEOS Y SUS TIPOS (Dataset para ML)
# ==============================================================================

# Definición de vídeos de ejemplo y sus etiquetas de tipo
simulated_videos_data = [
    {
        "name": "video_animales_1",
        "type": "Animales",
        "urls": image_urls_base[0:3] # Perro, gato, perro en agua
    },
    {
        "name": "video_vehiculos_1",
        "type": "Vehiculos",
        "urls": image_urls_base[3:6] # Tren, coche, coche (frontal)
    },
    {
        "name": "video_comida_1",
        "type": "Comida",
        "urls": image_urls_base[6:8] + [image_urls_base[6]] # Comida, pizza, comida
    },
    {
        "name": "video_animales_2",
        "type": "Animales",
        "urls": [image_urls_base[0], image_urls_base[1], image_urls_base[2], image_urls_base[1]] # Mas animales
    },
    {
        "name": "video_variado_1",
        "type": "Variado",
        "urls": [image_urls_base[0], image_urls_base[4], image_urls_base[6]] # Animal, coche, comida
    },
    {
        "name": "video_vehiculos_2",
        "type": "Vehiculos",
        "urls": [image_urls_base[3], image_urls_base[4], image_urls_base[5], image_urls_base[3]] # Mas vehiculos
    }
    # Añade más vídeos simulados aquí para un dataset más grande y diverso
]

video_inferences = [] # Aquí guardaremos {id_video, df_inferencias, tipo_video}

for video_info in simulated_videos_data:
    # 1. Descargar y aumentar fotogramas para el vídeo actual
    download_and_augment_frames(video_info["urls"], video_info["name"])

    # 2. Clasificar objetos en los fotogramas de este vídeo
    video_frames_path = os.path.join(base_frames_dir, video_info["name"])
    df_inferencias_video = classify_frames_in_folder(video_frames_path, all_imagenet_classes)

    # 3. Guardar las inferencias junto con el tipo de vídeo
    video_inferences.append({
        "video_id": video_info["name"],
        "df_inferencias": df_inferencias_video,
        "video_type": video_info["type"]
    })

print("\n✅ Generación de DataFrames de inferencia por vídeo completada.")

# ==============================================================================
# ⚙️ PARTE 5 - INGENIERÍA DE CARACTERÍSTICAS PARA CLASIFICACIÓN DE VÍDEOS
# ==============================================================================

def create_video_features(video_df, all_possible_classes_list):
    """
    Crea un vector de características de tamaño fijo para un vídeo.
    Calcula la "importancia" de cada clase basada en frecuencia y confianza media.

    Args:
        video_df (pd.DataFrame): DataFrame de inferencias de objetos de un solo vídeo.
        all_possible_classes_list (list): Lista exhaustiva de todas las clases que el
            clasificador de objetos puede detectar (ej. las 1000 de ImageNet).

    Returns:
        dict: Un diccionario donde las claves son las clases y los valores son su "importancia"
            en el vídeo. Si una clase no aparece, su importancia es 0.
    """
    features = {cls: 0.0 for cls in all_possible_classes_list}

    if video_df.empty:
        return features

    # Número total de fotogramas únicos en el vídeo (para normalización)
    total_unique_frames = video_df['fotograma'].nunique()
    if total_unique_frames == 0:
        return features  # Evitar división por cero si no hay fotogramas clasificados

    # Agrupar por clase detectada
    grouped_by_class = video_df.groupby('clase_detectada').agg(
        count=('clase_detectada', 'size'),          # Número de veces que aparece la clase
        sum_confidence=('confianza', 'sum')         # Suma de las confianzas para esa clase
    ).reset_index()

    for _, row in grouped_by_class.iterrows():
        cls = row['clase_detectada']
        count = row['count']
        sum_confidence = row['sum_confidence']

        # Calcular frecuencia normalizada
        freq_normalized = count / total_unique_frames

        # Calcular confianza media para esa clase
        avg_confidence = sum_confidence / count

        # Combinar frecuencia y confianza. Puedes experimentar con esta fórmula.
        # Aquí, simplemente el producto.
        importance = freq_normalized * avg_confidence

        # Asignar la importancia solo si la clase está en nuestra lista global de clases
        if cls in features:
            features[cls] = importance

    return features

# Generar el dataset final de características (X) y etiquetas (y) para el clasificador de vídeos
X_video_features = []
y_video_types = []

print("\n⚙️ Generando características agregadas para cada vídeo para el clasificador final...")
for video_info in video_inferences:
    features_dict = create_video_features(video_info["df_inferencias"], all_imagenet_classes)
    X_video_features.append(list(features_dict.values()))
    y_video_types.append(video_info["video_type"])

# Convertir a DataFrame de Pandas para el entrenamiento
X = pd.DataFrame(X_video_features, columns=all_imagenet_classes)
y = pd.Series(y_video_types, name="video_type")

print("\n📊 DataFrame de características de vídeos (primeras 5 filas y 5 columnas):")
if not X.empty:
    print(X.iloc[:, :min(5, X.shape[1])].head()) # Muestra solo las primeras 5 columnas o menos si hay menos
else:
    print("DataFrame de características X está vacío. No se pudieron generar características.")
print("\n🎯 Etiquetas de tipo de vídeo:")
print(y)

# ==============================================================================
# 🤖 PARTE 6 - ENTRENAMIENTO Y EVALUACIÓN DEL CLASIFICADOR DE VÍDEOS
# ==============================================================================

# Solo proceder si X no está vacío y tiene suficientes muestras para split
if not X.empty and len(X) > 1 and len(y.unique()) > 1: # Necesitas al menos 2 muestras y 2 clases para stratify
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    print(f"\nTotal de vídeos para entrenamiento: {len(X_train)}")
    print(f"Total de vídeos para prueba: {len(X_test)}")

    # Inicializar y entrenar el clasificador de tipo de vídeo
    video_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    print("\n💪 Entrenando el clasificador de tipo de vídeo...")
    video_classifier.fit(X_train, y_train)
    print("✅ Entrenamiento completado.")

    # Realizar predicciones en el conjunto de prueba
    y_pred = video_classifier.predict(X_test)

    # Evaluar el rendimiento del clasificador
    print("\n✅ Evaluación del Clasificador de Tipo de Vídeo:")
    print(f"Precisión (Accuracy): {accuracy_score(y_test, y_pred):.2f}")
    print("\nReporte de Clasificación:")
    print(classification_report(y_test, y_pred, zero_division=0))

else:
    print("\n⚠️ No hay suficientes datos o clases para entrenar y evaluar el clasificador de vídeos. Necesitas más vídeos simulados o reales con diferentes tipos.")



Total de clases de ImageNet cargadas: 1000

📥 Descargando y preparando fotogramas para video_animales_1...
  ❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Dog_standing_in_snow.jpg/1280px-Dog_standing_in_snow.jpg: 404 Not Found
  ❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Retriever_in_water.jpg/1280px-Retriever_in_water.jpg: 404 Not Found
✅ Preparación de fotogramas para video_animales_1 completada.
  🚀 Iniciando inferencia en 3 fotogramas en simulated_video_frames\video_animales_1...

📥 Descargando y preparando fotogramas para video_vehiculos_1...
  ❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Seaside_Railway_Station%2C_Ryde_Pier_Head_%28107774%29.jpg/1280px-Seaside_Railway_Station%2C_Ryde_Pier_Head_%28107774%29.jpg: 404 Not Found
  ❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Car_with_a_blue_sky.jpg/1280px-Car_with_a_blue_sky.jpg: 404 Not Found
  ❌ Error

ValueError: 1000 columns passed, passed data had 998 columns